In [1]:
import os
import leafmap
import rasterio
import rasterio
from rasterio.warp import transform_bounds


%cd "D:\_pribadi\PROJECT\AeroArb\"
HOME = os.getcwd()

# SAMPLE IMAGE
# image_path_tif = f"{HOME}/Map22_Orthomosaic_export_TueSep17092205633709_com.tif"
image_path_tif = f"{HOME}/sample gambar/Map1_Orthomosaic_export_FriJun23075900983961_com.tif"
image_tif_copy = f"{HOME}/image.tif"  # Path for the copied image
image_map = f"{HOME}/image_map.tif"  # Path for the copied image

# Function to copy the GeoTIFF file
def copy_geotiff(source_path, destination_path):
    with rasterio.open(source_path) as src:
        with rasterio.open(destination_path, 'w', driver='GTiff',
                           height=src.height, width=src.width,
                           count=src.count, dtype=src.dtypes[0],
                           crs=src.crs, transform=src.transform) as dst:
            src_crs = str(src.crs)
            print(src_crs)
            for i in range(1, src.count + 1):
                dst.write(src.read(i), i)

def is_valid_bbox(bbox):
    left, top, right, bottom = bbox
    return (-180 <= left <= 180 and -180 <= right <= 180 and
            -90 <= top <= 90 and -90 <= bottom <= 90)

def get_bbox_from_tif(image_path, dest_crs="EPSG:4326"):
    with rasterio.open(image_path) as src:
        # Dapatkan bounds asli dari file TIFF
        src_bounds = src.bounds
        src_crs = src.crs  # CRS asli dari file, seperti EPSG:27700 untuk British National Grid
        print(f"Transforming from {src_crs} to {dest_crs}")
        
        # Pastikan CRS asli benar (misalnya EPSG:27700), lalu transformasikan ke CRS tujuan
        transformed_bounds = transform_bounds(src_crs, dest_crs,
                                              src_bounds.left, src_bounds.bottom,
                                              src_bounds.right, src_bounds.top)
        return list(transformed_bounds)

def get_center_from_bbox(bbox):
    left, top, right, bottom = bbox
    center_lon = (left + right) / 2
    center_lat = (top + bottom) / 2
    return center_lon, center_lat

# Copy the GeoTIFF file
copy_geotiff(image_path_tif, image_tif_copy)

D:\_pribadi\PROJECT\AeroArb
EPSG:27700


## Create an interactive map (optional) for selected image sample


In [ ]:
bbox = get_bbox_from_tif(image_path_tif)
center = get_center_from_bbox(bbox)

# Verifikasi bbox sebelum melanjutkan
if is_valid_bbox(bbox):
    print(f'bbox: {bbox}')
    print(f'center: {center}')
else:
    print("Bounding box tidak valid:", bbox)

In [ ]:
# m = leafmap.Map(center=center, zoom=18, height="800px")
m = leafmap.Map(center=[-22.17615, -51.253043], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

In [ ]:
from segment_geospatial.samgeo import tms_to_geotiff

bounds = m.user_roi_bounds()
if bounds is None:
    bounds = bbox
    
tms_to_geotiff(output=image_map, bbox=bounds, zoom=19, source="Satellite", overwrite=True)
m.layers[-1].visible = False
m.add_raster(image_map, layer_name="Image")

image_tif_copy = image_map


## Tree Segmentations SAM2


In [2]:

import os
%cd "D:\_pribadi\PROJECT\AeroArb\"
HOME = os.getcwd()

import leafmap
from segment_geospatial_v2.samgeo.text_sam import LangSAM
import leafmap
import pandas as pd
import json

# USING CLASSES
classes = True

image_path_png = f"{HOME}/mask.png"
mask_path = f"{HOME}/trees.tif"
geojson_path = f"{HOME}/objects.geojson"
geojson_species_path = f"{HOME}/species.geojson"
object_tif_path = f"{HOME}/objects.tif"
object_csv_path = f"{HOME}/objects.csv"

m = leafmap.Map(center=[-22.17615, -51.253043], zoom=18, height="800px")
m.add_basemap("SATELLITE")

sam = LangSAM(model_type="sam2-hiera-large")

text_prompt = "tree"
results = sam.predict(image_tif_copy, text_prompt, box_threshold=0.24, text_threshold=0.24, return_results=True)
classifications = sam.generate_anns(
    predict_masks=True,
    add_boxes=False,
    blend=False,
    output=mask_path,
)
da, gdf = sam.region_groups(
    image=mask_path,
    min_size=100,
    out_csv=object_csv_path,
    out_image=object_tif_path,
    out_vector=geojson_path
)

species_list = classifications["species"]

csv_data = pd.read_csv(object_csv_path)
if len(species_list) != len(csv_data):
    species_list = species_list + ["unknown"] * (len(csv_data) - len(species_list))
    species_list = species_list[:len(csv_data)]
# Update each feature to add 'species' key after 'label'
insert_position = csv_data.columns.get_loc("label") + 1
csv_data.insert(insert_position, "species", species_list)
csv_data.to_csv(object_csv_path, index=False)

with open(geojson_path, 'r') as f:
    geojson_data = json.load(f)
# Update each feature to add 'species' key after 'label'
for species, feature in zip(species_list, geojson_data.get("features", [])):
    properties = feature.get("properties", {})
    if "label" in properties:
        # Insert 'species' after 'label'
        updated_properties = {}
        for key, value in properties.items():
            updated_properties[key] = value
            if key == "label":
                updated_properties["species"] = species  # Default value
        feature["properties"] = updated_properties
# Save the updated GeoJSON
with open(geojson_species_path, 'w') as f:
    json.dump(geojson_data, f, indent=2)


m.layers[-1].visible = False
m.add_raster(image_tif_copy, layer_name="Image")
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#5be271",
    "fillOpacity": 0.5,
}
m.add_vector(geojson_species_path, layer_name="Vector", style=style)
m

# RESTART SETELAH DIPAKAI

D:\_pribadi\PROJECT\AeroArb
final text_encoder_type: bert-base-uncased
Resized image to 150x147
Resized image to 150x117
Resized image to 150x184
Resized image to 150x158
Resized image to 150x134
Resized image to 150x142
Resized image to 150x157
Resized image to 150x163
Resized image to 150x148
Resized image to 150x146
Resized image to 150x132
Resized image to 150x134


Map(center=[52.4489475, 0.724175], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…